# Importing and saving Corals locations from "Coral reef potential connectivity in the southwest Indian Ocean."

From Noam S. Vogt-Vincent Github page: https://github.com/nvogtvincent/WIO_Coral/tree/main/SCRIPTS, it is possible to download the locations of the coral spots in the Western Indian Ocean region (in particular inside the domain of WINDS dataset) both in every cell of WINDS grid and grouped with a clusterization done by the team of the research.

- 'site_reference_cell.csv' contains the cell points
- 'site_reference_grp_reordered.xlsx' contains the grouped locations

In [1]:
import xarray as xr
import pandas as pd
import numpy as np

### Defining the domain (thesis case study)

In [2]:
inf_lat = -8.161049
sup_lat = -2.1883478
left_lon = 37.62
right_lon = 43.62
extent = [left_lon-0.1, right_lon+0.1, sup_lat+0.1, inf_lat-0.1]

### Opening and visualizing the locations inside the domain
#### Ex: cell locations

In [3]:
# First download the file and then name it as 'site_reference_cell.csv'
csvFile = pd.read_csv('site_reference_cell.csv')

# Select only the locations inside the domain
ds_cells_big = csvFile[((csvFile['Longitude'] >= left_lon) & (csvFile['Longitude'] < right_lon)) & ((csvFile['Latitude'] >= inf_lat) & (csvFile['Latitude'] < sup_lat))]

# Drop the variables not relevant (keeping only Latitude and Longitude)
ds_cells = ds_cells_big.drop(columns=['Unnamed: 0','Group number','L2 Group','L1 Group','Country','Reef area','Cell number'])

The same method for the grouped locations: xlsxFile = pd.read_excel('site_reference_grp_reordered.xlsx')

### Deleting points outside the domain of WINDS dataset (so on land)

Importing the velocity field (here v-field, but it is ok also u-field and mesh)

In [4]:
days70_from_april_1997_velocity_v_OK = xr.open_dataset("days70_from_april_1997_velocity_v_OK.nc")

# Just renaming for a better handling
v_surf = days70_from_april_1997_velocity_v_OK.v_surf[0,:,:].values
lon_v = days70_from_april_1997_velocity_v_OK.nav_lon_rho[0,:].values
lat_v = days70_from_april_1997_velocity_v_OK.nav_lat_rho[:,0].values

# Function to find the nearest index in a sorted array
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

# Filtering the locations
filtered_lon_list = []
filtered_lat_list = []

for lon, lat in zip(ds_cells['Longitude'].values, ds_cells['Latitude'].values):
    # Finding the nearest grid cell in the velocity field
    lon_idx = find_nearest(lon_v, lon)
    lat_idx = find_nearest(lat_v, lat)
    
    # Checking if the point is within the bounds of the velocity field
    if 0 <= lon_idx < len(lon_v) and 0 <= lat_idx < len(lat_v):
        # Checking if the velocity value at this location is non-zero
        if v_surf[lat_idx, lon_idx] != 0:
            filtered_lon_list.append(lon)
            filtered_lat_list.append(lat)

### Creating squares around each point (to later make particles start from these points)

In [5]:
# Square size (0.02 to the left, to the right, above and below every 1133 filtered points)
dx = 0.02
dy = 0.02

# Number of random points per square
num_points = 10

# Set random seed for reproducibility
np.random.seed(42)

# Latitude and Longitude of the 1133x10 points
random_latitudes = []
random_longitudes = []

# Generating random points within squares
for lon, lat in zip(filtered_lon_list, filtered_lat_list):
    lower_lat = lat - dy
    upper_lat = lat + dy
    left_lon = lon - dx
    right_lon = lon + dx

    # Generating random points and ensure they are within valid cells of the velocity field
    generated_points = 0
    while generated_points < num_points:
        random_lat = np.random.uniform(lower_lat, upper_lat)
        random_lon = np.random.uniform(left_lon, right_lon)
        
        # Finding the nearest indices in the velocity field grid
        lon_idx = find_nearest(lon_v, random_lon)
        lat_idx = find_nearest(lat_v, random_lat)
        
        # Checking if the point is within the bounds and in a non-zero velocity cell
        if 0 <= lon_idx < len(lon_v) and 0 <= lat_idx < len(lat_v) and v_surf[lat_idx, lon_idx] != 0:
            random_latitudes.append(random_lat)
            random_longitudes.append(random_lon)
            generated_points += 1

It is possible to save all these locations to have them on a csv/xlsx file to use them directly every time with np.save

### Grouped locations used for the connectivity matrices (thesis case study)

In [ ]:
new_corals_lon = np.array([39.73223114, 39.6405884, 39.500, 39.92, 39.47441864, 39.50928879,
        39.9375    , 39.56, 39.34, 39.26, 39.03000259, 38.9177742 ,
        39.13373184, 39.30393982, 39.60278183, 39.4683342 , 39.18, 39.42,
        38.88454819, 39.01739502, 39.10639954, 39.20000076, 39.26356888,
        39.6333313 , 39.86, 39.88, 39.7731514 , 39.64492798, 39.36883545,
        39.53882599, 39.67703629, 39.89500046, 40.05157852, 40.23666763,
        40.67300415, 40.98210526]) 

new_corals_lat = np.array([-8.01682186, -7.69152546, -7.99, -7.745, -7.48969936, -7.23591518,
        -6.89461374, -6.982, -6.809, -6.594,  -6.45777082, -6.18345165,
        -6.16138983, -6.36267281, -6.37120247, -6.0890007 , -5.877, -5.776,
        -5.79438543, -5.54667282, -5.28294134, -5.02995682, -4.80961895,
        -5.42941427, -5.304, -5.048 , -4.86327362, -5.17221212, -4.68640804,
        -4.43031645, -4.12676477, -3.63475347, -3.35852289, -2.85103106,
        -2.55003238, -2.24830103])